# **Recomendador de artículos usando vectorización de palabras (versión texto completo)**

El contenido es aprendizaje usando vectorización de palabras, modelo entrenado con contenido de Quien, Expansión, Obras, etc.

Colaborativo porque las recomendaciones están basadas en número de visitas de las notas recomendadas (se recomiendan notas que han tenido exito previo).

https://towardsdatascience.com/the-remarkable-world-of-recommender-systems-bff4b9cbe6a7


In [0]:
from google.colab import drive
drive.mount('/content/gdrive') # grant access to google drive filesystem

!pip install newspaper3k
from newspaper import Article 

!pip install pandas --upgrade
import pandas as pd

!pip install PyPrind
import pyprind

import spacy
from spacy.lang.es.stop_words import STOP_WORDS
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import os
from gensim.models import KeyedVectors

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.24.2)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#@title <-- 2. Presiona este botón para enviar la URL a analizar

#@markdown Copia y pega la URL del artículo a analizar en el siguiente campo:
url = "https://expansion.mx/economia/2019/06/06/trump-evalua-posponer-aranceles-contra-mexico-bloomberg" #@param {type:"string"}
#@markdown 
#@markdown El proceso continúa en la siguientes celdas...

# Obtener texto limpio de la url y un doc de Spacy:
 
article = Article(url)
article.download()
article.parse()
#print(article.publish_date)  # left for debugging
#print(article.text)          # left for debugging


In [0]:
# Initializations

notesFile = 'content.csv'
path = '/content/gdrive/My Drive/datos/'
pathWV = '/content/gdrive/My Drive/datos/verbs/' # path al Word Vector

stop_words = stopwords.words('spanish')
stop_words.extend(['desde', 'expansión', 'publicidad', 'adnpolítico', 'getty images', 'con', '¿'])

# Stop words, palabras a ignorar
stop_words = STOP_WORDS
stop_words.add('expansión')
stop_words.add('publicidad')
stop_words.add('adnpolítico')
stop_words.add('adnpolítico)')
stop_words.add('adnpolítico).-')
stop_words.add('getty images')
stop_words.add('méxico')
stop_words.add('con)')
stop_words.add('“')
stop_words.add('\n')
stop_words.add('¿')

# Stop lines, líneas a ignorar
stop_lines = set()
stop_lines.add('te puede interesar:')
stop_lines.add('Recomendamos:')
stop_lines.add('También Lee:')
stop_lines.add('Con información de Notimex')
stop_lines.add('Notimex')
stop_lines.add('Publicidad')
stop_lines.add('\n')

In [0]:
# Import Dataset
with open(os.path.join(path, notesFile)) as csv_file:
    dfPre = pd.read_csv(csv_file)

# Load word vectors
#wvAdnpolitico = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'adnpolitico_model.txt'), binary=False) 
wvExpansion = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'expansion_model.txt'), binary=False) 
#wvLifeandstyle = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'lifeandstyle_model.txt'), binary=False) 
#wvManufactura = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'manufactura_model.txt'), binary=False) 
#wvObras = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'obras_model.txt'), binary=False) 
#wvQuien = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'quien_model.txt'), binary=False) 
#wvTodos = KeyedVectors.load_word2vec_format(os.path.join(pathWV, 'todos_model.txt'), binary=False) 

In [0]:
# parámetros de búsqueda
percentage = 1.0  # percentage of dataset to be analyzed (1.0 = 100%)
start_date = pd.to_datetime('04-01-2019') # mes-dia-año 🙄
end_date = pd.to_datetime('04-30-2019')
maxDistance = 0.75 # Distancia entre el texto provisto y los recomendados (parámetro obsoleto, mejor mostramos los primeros nResults)
sitio = 'Expansión' # Sitio en donde hacer la busqueda (debe coincidir con el modelo, ejemplo: wvExpansion.init_sims)
seccion = 'empresas' # Seccion del sitio 
nResults = 23 # Number of rows to show


# Comentar la línea que no se quiera usar como criterio
df = dfPre.loc[dfPre['texto'] == dfPre['texto']] # Eliminar registros con Nulos en 'texto'
df = df.loc[df['url'] == df['url']] # Eliminar registros con nulos en'url'
#df = df.loc[df['sitio'] == 'Expansión'] # Seleccionar sitio 
#df = df.loc[df['seccion'] == seccion] # Seleccionar seccion (Puede omitirse)
df = df.loc[(pd.to_datetime(df['fecha']) >= start_date) & (pd.to_datetime(df['fecha']) <= end_date)]
df = df.loc[df['pageviews'] >= df.loc[:,"pageviews"].median()] # mayor que la mediana de los pageviews

if len(df) > 0:
    df.set_index('url')
    oneText = article.text.lower().split()
    dfResult = pd.DataFrame()
    cols = ['dist', 'pv', 'url']
    wvExpansion.init_sims(replace=True)
    df = df.sample(frac=percentage)
    count = len(df) 
    perc = pyprind.ProgPercent(count)
    for index, row in df.iterrows(): # for each article in dataset
        texto = row['texto'].lower().split()
        distance = wvExpansion.wmdistance(texto, oneText)
        #if distance <= maxDistance and distance > 0:
        dfResult = dfResult.append(pd.DataFrame({'dist': distance, 'pv': row['pageviews'], 'url': row['url']}, index=[0]))
        perc.update() # signal to increment the progress %
    if len(dfResult) > 0:
        print("\n")
        print(*dfResult.sort_values(by='dist').head(nResults).values.tolist(),sep = "\n" )
    else:
        print("\n")
        print('No se encontraron sugerencias. Puedes intentar cambiando los parámetros')    
else:
    print("\n")
    print('No se encontraron sugerencias. Puedes intentar cambiando los parámetros')


[100 %] Time elapsed: 00:09:08 | ETA: 00:00:00



[0.6085923237401172, 2524, 'https://adnpolitico.com/presidencia/2019/04/08/amlo-amenaza-con-cancelar-tanto-su-reforma-educativa-como-la-de-pena-nieto']
[0.6147692926568319, 6970, 'https://adnpolitico.com/presidencia/2019/04/12/amlo-y-jorge-ramos-protagonizan-un-debate-en-plena-mananera']
[0.6149189440503927, 2194, 'https://expansion.mx/economia/2019/04/03/importadores-y-exportadores-tienen-plan-b-en-caso-de-cierre-de-frontera-con-eu']
[0.6208995960081745, 1146, 'https://expansion.mx/economia/2019/04/26/la-coparmex-pide-a-amlo-seguir-con-zee-y-el-cce-respeto-a-los-contratos']
[0.6251634898354917, 28718, 'https://expansion.mx/economia/2019/04/09/amlo-lanza-advertencia-a-gasolineros-y-considera-crear-estaciones-de-servicio']
[0.6278275719933488, 909, 'https://adnpolitico.com/presidencia/2019/04/20/amlo-llama-hipocritas-a-criticos-de-su-memorandum-contra-reforma-educativa']
[0.6319509501700876, 2387, 'https://adnpolitico.com/presidencia/2019/04/09/amlo-desmiente-otra-vez-a-subsecretario-


Total time elapsed: 00:09:08


In [0]:
dfResult

,dist,pv,url
0,0.805432,34011,https://quien.com/espectaculos/2019/04/19/rode...
0,0.764030,44290,https://quien.com/realeza/2019/04/02/alto-prec...
0,0.839832,885,https://lifeandstyle.mx/estilo/2019/04/22/espe...
0,0.701792,31541,https://quien.com/espectaculos/2019/04/06/labi...
0,0.695657,1927,https://adnpolitico.com/mexico/2019/04/14/el-i...
0,0.787042,1967,https://expansion.mx/empresas/2019/04/26/sfp-i...
0,0.726443,937,https://expansion.mx/tecnologia/2019/04/04/elo...
0,0.744061,1363,https://lifeandstyle.mx/entretenimiento/2019/0...
0,0.703191,1400,https://expansion.mx/carrera/2019/04/25/ahora-...
0,0.712037,5272,https://quien.com/realeza/2019/04/17/senales-q...
